- [ ] Ler todos os arquivos
- [ ] Separar por municipios
- [ ] Aplicar funcao de limpeza
- [ ] Salvar no banco

In [11]:
PASTA = './../../dados'

# Ler municipios

In [12]:
import duckdb
import pandas as pd

def ler_municipios(pasta_csvs):
    query = f"""
        SELECT DISTINCT estado, municipio
        FROM read_csv_auto('{pasta_csvs}/*.csv', union_by_name=true)
        WHERE estado IS NOT NULL AND municipio IS NOT NULL
        ORDER BY estado, municipio 
    """
    return duckdb.query(query).to_df()

In [13]:
municipios_df = ler_municipios(PASTA)

len(municipios_df)

5570

# Ler os .csvs filtrando por municipio e estado

In [14]:
import duckdb
import pandas as pd

def ler_csvs_filtrando_municipio(pasta_csvs, municipio, estado, nome_coluna_municipio='municipio', nome_coluna_estado='estado'):
    query = f"""
        SELECT *
        FROM read_csv_auto('{pasta_csvs}/*.csv', union_by_name=true)
        WHERE {nome_coluna_municipio} = '{municipio}' and {nome_coluna_estado} = '{estado}'
    """
    return duckdb.query(query).to_df()

# Funçao de limpeza

In [15]:
def suavizar(df, window_size=3, threshold=2):
    """
    Smooth data by replacing outliers with previous day's casosNovos.
    
    Parameters:
    - df: pandas DataFrame with columns 'date' and 'casosNovos'
    - window_size: number of days to consider in rolling window
    - threshold: number of standard deviations to use for outlier detection
    
    Returns:
    - DataFrame with smoothed values
    """
    # Make a copy to avoid modifying original data
    df_smoothed = df.copy()
    
    # Calculate rolling statistics
    rolling_mean = df['casosNovos'].rolling(window=window_size, center=True, min_periods=1).mean()
    rolling_std = df['casosNovos'].rolling(window=window_size, center=True, min_periods=1).std()
    
    # Identify outliers (values outside mean ± threshold*std)
    lower_bound = rolling_mean - threshold * rolling_std
    upper_bound = rolling_mean + threshold * rolling_std
    
    is_outlier = (df['casosNovos'] < lower_bound) | (df['casosNovos'] > upper_bound)
    
    # Replace outliers with previous day's value
    df_smoothed['casosNovos'] = df['casosNovos'].where(~is_outlier, df['casosNovos'].shift(1))
    
    # For the first row (no previous value), use the next value if available
    if is_outlier.iloc[0] and len(df) > 1:
        df_smoothed.iloc[0, df_smoothed.columns.get_loc('casosNovos')] = df['casosNovos'].iloc[1]
    
    return df_smoothed

# Recalcula casos acumulados

In [16]:
def recalcula_casos_acumulados(df):
  df['novos_casos_acumulados'] = df['casosNovos'].cumsum()
  return df

# Função de limpeza

In [17]:
def limpar(df):
  df = suavizar(df)
  df = recalcula_casos_acumulados(df)
  return df

In [18]:
OUTPUT_FOLDER = 'saida/'

for i in range(len(municipios_df)):
  municipio = municipios_df.at[i, 'municipio']
  estado = municipios_df.at[i, 'estado']
  df = ler_csvs_filtrando_municipio(PASTA, municipio, estado)
  df = limpar(df)

  df.to_csv(f'saida/{estado}_{municipio}.csv')
  print(f"{estado}_{municipio} salvos")

AC_Acrelândia salvos
AC_Assis Brasil salvos
AC_Brasiléia salvos
AC_Bujari salvos
AC_Capixaba salvos
AC_Cruzeiro do Sul salvos
AC_Epitaciolândia salvos
AC_Feijó salvos
AC_Jordão salvos
AC_Manoel Urbano salvos
AC_Marechal Thaumaturgo salvos
AC_Mâncio Lima salvos
AC_Plácido de Castro salvos
AC_Porto Acre salvos
AC_Porto Walter salvos
AC_Rio Branco salvos
AC_Rodrigues Alves salvos
AC_Santa Rosa do Purus salvos
AC_Sena Madureira salvos
AC_Senador Guiomard salvos
AC_Tarauacá salvos
AC_Xapuri salvos
AL_Anadia salvos
AL_Arapiraca salvos
AL_Atalaia salvos
AL_Barra de Santo Antônio salvos
AL_Barra de São Miguel salvos
AL_Batalha salvos
AL_Belo Monte salvos
AL_Belém salvos
AL_Boca da Mata salvos
AL_Branquinha salvos
AL_Cacimbinhas salvos
AL_Cajueiro salvos
AL_Campestre salvos
AL_Campo Alegre salvos
AL_Campo Grande salvos
AL_Canapi salvos
AL_Capela salvos
AL_Carneiros salvos
AL_Chã Preta salvos
AL_Coité do Nóia salvos
AL_Colônia Leopoldina salvos
AL_Coqueiro Seco salvos
AL_Coruripe salvos
AL_Craíb

ParserException: Parser Error: syntax error at or near "Água"